In [ ]:
import random
import csv
import logging

from shop_crawler import *
from selenium_helper import *
from status import *

import trace_logger
import common_actors

from contextlib import contextmanager

### Prepare different urls to analyze

In [ ]:
# All urls
all_urls = []
with open('../resources/pvio_vio_us_ca_uk_sample1.csv', 'r') as f:
    rows = csv.reader(f)
    for row in rows:
        url = row[0]
        if url:
            all_urls.append(url)

# Random sample urls
random.seed(1)
sample_urls = random.sample(all_urls, 100)

# Some good urls to analyze by hands
good_urls = [
    'vape-fuel.com',
    'theglamourshop.com',
    'firstfitness.com',
    'sandlakedermatology.com',
    'getwaave.com',
    'dixieems.com',
    'jonessurgical.com',
    'srandd.com',
    'ambarygardens.com',
    'anabolicwarfare.com'
]


### Mock user and payment information

In [ ]:
user_info = UserInfo(
    first_name = 'John',
    last_name = 'Smith',
    country = 'United States',
    home = 34,
    street = 'Ocean drive',
    city = 'Miami',
    zip = '33125',
    state = 'FLorida',
    
    phone = '1231232',
    email = 'john@service.com'
)

billing_info = PaymentInfo(
    card_number = '1413232312312321',
    card_name = 'Visa Card',
    card_type = 'Visa',
    expire_date_year = 2020,
    expire_date_month = 12,
    cvc = '123'
)

In [ ]:
selenium_path = '/usr/bin/chromedriver'

@contextmanager
def get_crawler(headless = True):
    logger = trace_logger.FileTraceLogger('results.jsonl', 'images')

    global user_info, billing_info, selinium_path
    crawler = ShopCrawler(user_info, 
                          billing_info, 
                          selenium_path, 
                          headless = headless, 
                          trace_logger = logger)
    common_actors.add_crawler_extensions(crawler)
    
    yield crawler

def get_driver(headless = True):
    global selenium_path   
    return create_chrome_driver(selenium_path, headless = headless)


### Set up logging level

In [ ]:
logger = logging.getLogger('shop_crawler')
logger.setLevel(logging.DEBUG)

handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

### Start crawling for every url

In [ ]:
results = []

with get_crawler(False) as crawler:
    for url in good_urls:
        print('\n\n{}'.format(url))
        status = crawler.crawl(url, attempts=1)
        print(status)
        results.append(status)


In [ ]:
states = {}
for status in results:
    if isinstance(status, ProcessingStatus):
        states[status.state] = states.get(status.state, 0) + 1
        
print(states)

In [ ]:
import json
from IPython.core.display import Image, display

with open('results.jsonl', 'r') as f:
    for line in f:
        trace = json.loads(line)
        print('\n\n{} - {}'.format(trace['domain'], trace['status']['state']))
        for step in trace['steps']:
            print('url: {} state: {} handler: {}'.format(step['url'], step['state'], step['handler']))
            display(Image(step['screen_path'], width=640, unconfined=True))